Here is my new codespace and such to work on this Excel Sheet

In [1]:
pip install pandas numpy matplotlib seaborn scipy statsmodels wbgapi openpyxl


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import os

# Update this to 'AFRICA_DATA' if there is no .csv extension in the sidebar
file_name = 'AFRICA_DATA.xlsx'

if os.path.exists(file_name):
    df = pd.read_excel(file_name)
    print(f"File '{file_name}' loaded successfully.")
    print(df.head(10))
else:
    print(f"File '{file_name}' not found. Current files in directory:")
    print(os.listdir('.'))

File 'AFRICA_DATA.xlsx' loaded successfully.
   Country  Year Primary_Region  Yield  Rainfall_mm  Lagged_Rainfall  \
0  Algeria  1970            AMU  637.5         89.0              NaN   
1  Algeria  1971            AMU  587.8         89.0             89.0   
2  Algeria  1972            AMU  671.7         89.0             89.0   
3  Algeria  1973            AMU  493.7         89.0             89.0   
4  Algeria  1974            AMU  497.9         89.0             89.0   
5  Algeria  1975            AMU  845.2         89.0             89.0   
6  Algeria  1976            AMU  690.5         89.0             89.0   
7  Algeria  1977            AMU  410.9         89.0             89.0   
8  Algeria  1978            AMU  586.1         89.0             89.0   
9  Algeria  1979            AMU  561.5         89.0             89.0   

   GDP_per_Capita ISO3  is_AMU  is_ECOWAS  is_EAC  is_ECCAS  is_COMESA  \
0      352.845767  DZA       1          0       0         0          0   
1      360.111

In [3]:
import wbgapi as wb

# Define the indicators for Inflation and Income Level (GNI per capita)
indicators = {
    'FP.CPI.TOTL.ZG': 'Inflation_Annual_Percent',
    'NY.GNP.PCAP.CD': 'GNI_per_Capita'
}

# Get the list of unique ISO3 country codes from your loaded data
country_codes = df['ISO3'].unique().tolist()

print("Fetching new variables from World Bank...")
# Fetch the data
new_data = wb.data.DataFrame(indicators.keys(), country_codes, time=range(1970, 2024), numericTimeKeys=True)

# Reshape the data to match your current format (Country/Year rows)
new_data = new_data.stack().unstack(level=1).reset_index()
new_data.columns = ['ISO3', 'Year'] + list(indicators.values())

# Merge the new variables into your main DataFrame
df = pd.merge(df, new_data, on=['ISO3', 'Year'], how='left')

print("New variables merged successfully!")
print(df[['Country', 'Year', 'Inflation_Annual_Percent', 'GNI_per_Capita']].tail(10))

Fetching new variables from World Bank...
New variables merged successfully!
       Country  Year  Inflation_Annual_Percent  GNI_per_Capita
2636  Zimbabwe  2014                 -0.197785          1270.0
2637  Zimbabwe  2015                 -2.430968          1330.0
2638  Zimbabwe  2016                 -1.543670          1240.0
2639  Zimbabwe  2017                  0.893962          1680.0
2640  Zimbabwe  2018                 10.618866          2140.0
2641  Zimbabwe  2019                255.304991          2430.0
2642  Zimbabwe  2020                557.201817          1980.0
2643  Zimbabwe  2021                 98.546105          2320.0
2644  Zimbabwe  2022                104.705171          2580.0
2645  Zimbabwe  2023                       NaN          2550.0


In [4]:
import pandas as pd
import wbgapi as wb

# 1. Load your existing dataset
file_name = 'AFRICA_DATA.xlsx'
df = pd.read_excel(file_name)

# 2. Define indicators: 
# FP.CPI.TOTL = Consumer Price Index (Price levels)
# AG.PRD.FOOD.XD = Food Production Index (Supply levels)
indicators = {
    'FP.CPI.TOTL': 'Food_Price_Index_CPI',
    'AG.PRD.FOOD.XD': 'Food_Production_Index'
}

# 3. Fetch the data for your specific countries and years
print("Fetching food price and production data...")
country_codes = df['ISO3'].unique().tolist()
new_data = wb.data.DataFrame(indicators.keys(), country_codes, time=range(1970, 2024), numericTimeKeys=True)

# 4. Reshape and Merge
new_data = new_data.stack().unstack(level=1).reset_index()
new_data.columns = ['ISO3', 'Year'] + list(indicators.values())

# Merge to the right of your current dataset
df = pd.merge(df, new_data, on=['ISO3', 'Year'], how='left')

# 5. Save the updated file
df.to_excel('AFRICA_DATA_UPDATED.xlsx', index=False)

print("Success! New columns added to the right.")
print(df[['Country', 'Year', 'Yield', 'Food_Price_Index_CPI', 'Food_Production_Index']].tail(10))

Fetching food price and production data...
Success! New columns added to the right.
       Country  Year   Yield  Food_Price_Index_CPI  Food_Production_Index
2636  Zimbabwe  2014   831.4                 96.85             108.859485
2637  Zimbabwe  2015   557.5                101.10             106.213145
2638  Zimbabwe  2016   435.1                102.05             104.573565
2639  Zimbabwe  2017  1203.3                106.59             105.508414
2640  Zimbabwe  2018  1254.2                107.82             116.712211
2641  Zimbabwe  2019   749.6                105.74             414.684309
2642  Zimbabwe  2020  1149.4                110.34            2725.312815
2643  Zimbabwe  2021  1545.5                120.52            5411.002445
2644  Zimbabwe  2022   956.9                121.68           11076.601788
2645  Zimbabwe  2023   743.9                   NaN                    NaN


In [8]:
import wbgapi as wb

# Search for indicators containing "temperature"
print("Searching for temperature indicators...")
results = wb.series.info(q='temperature')

# If that returns nothing or not what you need, search specifically in the ESG database (Source 75)
print("\n--- Checking ESG Database (Source 75) ---")
try:
    wb.db = 75 # Switch to ESG database
    print(wb.series.info(q='temperature'))
except:
    print("Could not search Source 75.")

# Reset database to default
wb.db = 2

Searching for temperature indicators...

--- Checking ESG Database (Source 75) ---
id              value
--------------  ------------------------
EN.LND.LTMP.DC  Land Surface Temperature
                1 elements


In [12]:
import pandas as pd
import wbgapi as wb

# 1. SETUP & LOAD EXISTING COUNTRIES
file_path = 'AFRICA_DATA_UPDATED.xlsx'
indicator_code = 'EN.LND.LTMP.DC'  # Land Surface Temperature
source_id = 75                     # ESG Database

try:
    print(f"Reading country codes from {file_path}...")
    existing_df = pd.read_excel(file_path)
    
    # Create the list of countries from your unique 'ISO3' column
    iso3_codes = existing_df['ISO3'].unique().tolist()
    print(f"Found {len(iso3_codes)} unique countries to fetch data for.")

    # 2. FETCH DATA
    print(f"Fetching Temperature data ({indicator_code}) from Source {source_id}...")
    
    # Fetch data for the countries found in your file
    df_temp = wb.data.DataFrame(indicator_code, 
                                iso3_codes, 
                                time=range(2000, 2024), 
                                db=source_id)

    # 3. CLEAN & FORMAT
    df_temp.reset_index(inplace=True)
    
    # Melt to long format (Country, Year, Value)
    df_melted = df_temp.melt(id_vars=['economy'], var_name='Year', value_name='Land_Surface_Temp_C')
    
    # Clean Year column and Rename Country column to match your file
    df_melted['Year'] = df_melted['Year'].astype(str).str.replace('YR', '').astype(int)
    df_melted.rename(columns={'economy': 'ISO3'}, inplace=True)
    
    print(f"Data fetched! Found {len(df_melted)} rows. Merging now...")

    # 4. MERGE & SAVE
    # Merge existing data with new temperature data
    merged_df = pd.merge(existing_df, df_melted, on=['ISO3', 'Year'], how='left')

    merged_df.to_excel(file_path, index=False)
    
    print(f"Success! Temperature data added to {file_path}")
    print(merged_df[['ISO3', 'Year', 'Land_Surface_Temp_C']].head())

except Exception as e:
    print(f"Error: {e}")

Reading country codes from AFRICA_DATA_UPDATED.xlsx...
Found 48 unique countries to fetch data for.
Fetching Temperature data (EN.LND.LTMP.DC) from Source 75...
Data fetched! Found 1152 rows. Merging now...
Success! Temperature data added to AFRICA_DATA_UPDATED.xlsx
Error: "['Land_Surface_Temp_C'] not in index"


In [14]:
import pandas as pd

# Load the file
df = pd.read_excel('AFRICA_DATA_UPDATED.xlsx')

# Print all column names
print("Columns in file:")
print(df.columns.tolist())

# Print the first 2 rows to see what the data looks like
print("\nFirst 2 rows:")
print(df.head(2))

Columns in file:
['Country', 'Year', 'Primary_Region', 'Yield', 'Rainfall_mm', 'Lagged_Rainfall', 'GDP_per_Capita', 'ISO3', 'is_AMU', 'is_ECOWAS', 'is_EAC', 'is_ECCAS', 'is_COMESA', 'Lagged_Yield', 'Yield_Cereal_General', 'Crop_Production_Index', 'Agri_GDP_Share', 'Food_Insecurity_Prevalence', 'Food_Price_Index_CPI', 'Food_Production_Index', 'Land_Surface_Temp_C_x', 'Land_Surface_Temp_C_y']

First 2 rows:
   Country  Year Primary_Region  Yield  Rainfall_mm  Lagged_Rainfall  \
0  Algeria  1970            AMU  637.5         89.0              NaN   
1  Algeria  1971            AMU  587.8         89.0             89.0   

   GDP_per_Capita ISO3  is_AMU  is_ECOWAS  ...  is_COMESA  Lagged_Yield  \
0      352.845767  DZA       1          0  ...          0           NaN   
1      360.111082  DZA       1          0  ...          0         637.5   

   Yield_Cereal_General  Crop_Production_Index  Agri_GDP_Share  \
0                 26.33                  637.5             NaN   
1               

Combining Temp C x & Y

In [15]:
import pandas as pd

file_path = 'AFRICA_DATA_UPDATED.xlsx'
df = pd.read_excel(file_path)

print("Cleaning up duplicate columns...")

# 1. Create one master column
# We prioritize the new data (_y). If _y is empty, we keep the old data (_x).
if 'Land_Surface_Temp_C_y' in df.columns and 'Land_Surface_Temp_C_x' in df.columns:
    df['Land_Surface_Temp_C'] = df['Land_Surface_Temp_C_y'].fillna(df['Land_Surface_Temp_C_x'])
    
    # 2. Drop the _x and _y versions
    df.drop(columns=['Land_Surface_Temp_C_x', 'Land_Surface_Temp_C_y'], inplace=True)
    print("Merged _x and _y into a single 'Land_Surface_Temp_C' column.")

# 3. Save the clean file
df.to_excel(file_path, index=False)

print("Success! File is clean.")
print(df[['ISO3', 'Year', 'Land_Surface_Temp_C']].head())
print("\nCheck a recent year (2015) to see real data:")
print(df[df['Year'] == 2015][['ISO3', 'Year', 'Land_Surface_Temp_C']].head())

Cleaning up duplicate columns...
Merged _x and _y into a single 'Land_Surface_Temp_C' column.
Success! File is clean.
  ISO3  Year  Land_Surface_Temp_C
0  DZA  1970                  NaN
1  DZA  1971                  NaN
2  DZA  1972                  NaN
3  DZA  1973                  NaN
4  DZA  1974                  NaN

Check a recent year (2015) to see real data:
    ISO3  Year  Land_Surface_Temp_C
45   DZA  2015            37.687921
99   AGO  2015            29.936733
153  BEN  2015            33.468313
207  BFA  2015            36.062554
261  BDI  2015            28.188727


Splitting the Code into 7 Sheets. 1 Main, 5 Regional, 1 Variable Discription

In [16]:
import pandas as pd

file_path = 'AFRICA_DATA_UPDATED.xlsx'
df = pd.read_excel(file_path)

print("Creating regional sheets and metadata table...")

# 1. DEFINE THE METADATA (Variable Descriptions)
# We create a dictionary to explain what each column means
metadata_data = {
    'Variable Name': [
        'ISO3', 'Country', 'Year', 
        'Yield', 'Rainfall_mm', 'Lagged_Rainfall',
        'Land_Surface_Temp_C', 
        'Food_Price_Index_CPI', 'Food_Production_Index', 
        'Inflation_Annual_Percent', 'GNI_per_Capita', 'GDP_per_Capita', 'Agri_GDP_Share',
        'Food_Insecurity_Prevalence',
        'Primary_Region', 'is_AMU', 'is_ECOWAS', 'is_EAC', 'is_ECCAS', 'is_COMESA'
    ],
    'Description': [
        'Unique 3-letter Country Code (World Bank Standard)', 'Name of the Nation', 'Calendar Year of Observation',
        'Cereal Yield (kg per hectare)', 'Average Annual Precipitation (mm)', 'Rainfall from the previous year (to measure lagged effects)',
        'Average Land Surface Temperature (Celsius) - Source 75',
        'Consumer Price Index for Food (Base Year 2010 = 100)', 'Volume of Food Production (Base Year 2014-2016 = 100)',
        'Annual Inflation Rate (%)', 'Gross National Income per Capita (USD)', 'Gross Domestic Product per Capita (USD)', 'Agriculture Value Added as % of GDP',
        'Prevalence of Severe Food Insecurity (% of Population)',
        'Primary African Regional Economic Community (REC)', 'Binary Flag (1=Member of Arab Maghreb Union)', 'Binary Flag (1=Member of ECOWAS)', 
        'Binary Flag (1=Member of East African Community)', 'Binary Flag (1=Member of ECCAS)', 'Binary Flag (1=Member of COMESA)'
    ],
    'Source / Notes': [
        'World Bank', 'World Bank', 'Time Series',
        'World Bank (AG.YLD.CREL.KG)', 'Climate Data', 'Calculated',
        'World Bank ESG (EN.LND.LTMP.DC)',
        'World Bank (FP.CPI.TOTL)', 'World Bank (AG.PRD.FOOD.XD)',
        'World Bank (FP.CPI.TOTL.ZG)', 'World Bank (NY.GNP.PCAP.CD)', 'World Bank', 'World Bank',
        'FAO / World Bank',
        'Categorical', 'Binary (0/1)', 'Binary (0/1)', 'Binary (0/1)', 'Binary (0/1)', 'Binary (0/1)'
    ]
}

df_metadata = pd.DataFrame(metadata_data)

# 2. WRITE EVERYTHING TO EXCEL
# We use ExcelWriter to save multiple sheets into one file
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    
    # Sheet 1: The Master Data (All Countries)
    df.to_excel(writer, sheet_name='MASTER_DATA', index=False)
    
    # Sheet 2: AMU
    df[df['is_AMU'] == 1].to_excel(writer, sheet_name='AMU_Region', index=False)
    
    # Sheet 3: ECOWAS
    df[df['is_ECOWAS'] == 1].to_excel(writer, sheet_name='ECOWAS_Region', index=False)
    
    # Sheet 4: EAC
    df[df['is_EAC'] == 1].to_excel(writer, sheet_name='EAC_Region', index=False)
    
    # Sheet 5: ECCAS
    df[df['is_ECCAS'] == 1].to_excel(writer, sheet_name='ECCAS_Region', index=False)
    
    # Sheet 6: COMESA
    df[df['is_COMESA'] == 1].to_excel(writer, sheet_name='COMESA_Region', index=False)
    
    # Sheet 7: Variable Descriptions
    df_metadata.to_excel(writer, sheet_name='Variable_Descriptions', index=False)

print(f"Success! {file_path} now has 7 sheets organized by region + metadata.")

Creating regional sheets and metadata table...
Success! AFRICA_DATA_UPDATED.xlsx now has 7 sheets organized by region + metadata.


In [17]:
import pandas as pd

file_path = 'AFRICA_DATA_UPDATED.xlsx'

# Open the Excel file wrapper to see sheet names without loading all data
xls = pd.ExcelFile(file_path)

print("Sheets found in your file:")
for sheet in xls.sheet_names:
    print(f"- {sheet}")

Sheets found in your file:
- MASTER_DATA
- AMU_Region
- ECOWAS_Region
- EAC_Region
- ECCAS_Region
- COMESA_Region
- Variable_Descriptions
